In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import os
import keras
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.applications.mobilenet import preprocess_input
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dropout, Flatten, Dense, Activation
from tensorflow.python.keras.layers import Convolution2D, MaxPooling2D
from tensorflow.python.keras import backend as k
from keras.preprocessing import image
from mlxtend.evaluate import confusion_matrix
import matplotlib.image as mpimg

In [2]:
data_train = "datos/entrenamiento" 
data_test =  "datos/validacion"

### Definimos parametros de la cnn

In [3]:
epoch = 30
longitud, altura = 200, 200
batch_size = 32
filtros_conv1 = 8
filtros_conv2 = 16
filtros_conv3 = 32
tamaño_filtro1 = (8, 8)
tamaño_filtro2 = (4, 4)
tamaño_filtro3 = (2, 2)
max_pol = (2, 2)
clase = 2


### Preprocesamiento de la data

In [4]:
entrenamiento_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.3,
    zoom_range = 0.3,
    horizontal_flip = True)
validacion_datagen = ImageDataGenerator(1./255)
imagen_entrenamiento = entrenamiento_datagen.flow_from_directory(
   data_train,
    target_size = (altura, longitud),
    batch_size = batch_size,
    class_mode = 'categorical')
imagen_validacion = entrenamiento_datagen.flow_from_directory(
   data_test,
    target_size = (altura, longitud),
    batch_size = batch_size,
    class_mode = 'categorical')
print(imagen_entrenamiento.class_indices)

Found 12034 images belonging to 2 classes.
Found 2935 images belonging to 2 classes.
{'gato': 0, 'perro': 1}


In [5]:
pasos_entrenamiento = imagen_entrenamiento.n//imagen_entrenamiento.batch_size
pasos_validacion = imagen_validacion.n//imagen_validacion.batch_size

## Creación de la red convolucional

### Creación de las capas

In [6]:
cnn = Sequential()
cnn.add(Convolution2D(filtros_conv1, tamaño_filtro1, padding = 'same', input_shape = (longitud, altura, 3), activation = 'relu'))
cnn.add(MaxPooling2D(pool_size = max_pol))
cnn.add(Convolution2D(filtros_conv2, tamaño_filtro2, padding = 'same'))
cnn.add(MaxPooling2D(pool_size = max_pol))
cnn.add(Convolution2D(filtros_conv3, tamaño_filtro3, padding = 'same'))
cnn.add(Flatten())

cnn.add(Dense(220, activation = 'relu'))
cnn.add(Dense(120, activation = 'relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(clase, activation = 'sigmoid'))

### Compilacion

In [ ]:
cnn.compile(loss = 'categorical_crossentropy',
           optimizer = 'adam',
           metrics = ['accuracy'])
history = cnn.fit_generator(
    imagen_entrenamiento,
    steps_per_epoch = pasos_entrenamiento,
    epochs = epoch,
    validation_data = imagen_validacion,
    validation_steps = pasos_validacion
    )
#Prueba 3: 0.8451 
#Prueba 4: 0.8468

C:\Users\Jose1\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1969: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
376/376 [==============================] - 252s 669ms/step - loss: 0.7331 - accuracy: 0.5674 - val_loss: 0.6959 - val_accuracy: 0.5508
Epoch 2/30
376/376 [==============================] - 242s 643ms/step - loss: 0.6409 - accuracy: 0.6266 - val_loss: 0.6426 - val_accuracy: 0.6288
Epoch 3/30
376/376 [==============================] - 296s 786ms/step - loss: 0.6201 - accuracy: 0.6526 - val_loss: 0.5904 - val_accuracy: 0.6916
Epoch 4/30
376/376 [==============================] - 263s 699ms/step - loss: 0.6062 - accuracy: 0.6685 - val_loss: 0.5951 - val_accuracy: 0.7054
Epoch 5/30
376/376 [==============================] - 250s 666ms/step - loss: 0.5767 - accuracy: 0.7035 - val_loss: 0.5779 - val_accuracy: 0.7012
Epoch 6/30
376/376 [==============================] - 240s 638ms/step - loss: 0.5580 - accuracy: 0.7169 - val_loss: 0.5774 - val_accuracy: 0.7050
Epoch 7/30
376/376 [==============================] - 237s 630ms/step - loss: 0.5451 - accuracy: 0.7250 - val_loss: 0.5135 -

In [ ]:
dir='./modelo'

In [ ]:
if not os.path.exists(dir):
    os.mkdir(dir)
cnn.save('./modelo/modelo5.h5')
cnn.save_weights('./modelo/pesos5.h5')

In [ ]:
history_dict = history.history
print(history_dict.keys())

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Precision del modelo')
plt.ylabel('Precision')
plt.xlabel('epocas')
plt.legend(['Entrenamiento', 'test'], loc='upper left')
plt.show()
# hacemos las grafica 2 perdidas
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Perdidas del modelo')
plt.ylabel('perdidas')
plt.xlabel('epocas')
plt.legend(['Entrenamiento', 'test'], loc='upper left')
plt.show()